In [1]:
# utils.py
import os
import random
import numpy as np
import torch

def set_state(seed=42069):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

In [21]:
# config.py
from torchvision import datasets, transforms

dataset_to_transform = {
    'mnist': {
        'train': transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307), (0.3081))
        ]),
        'val': transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307), (0.3081))
        ]),
    },
    'cifar10': {
        'train': transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ]),
    },
    'cifar100': {
        'train': transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.507, 0.487, 0.441],
                                 std=[0.267, 0.256, 0.276])
        ]),
        'val': transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.507, 0.487, 0.441],
                                 std=[0.267, 0.256, 0.276])
        ]),
    }
}

train_ds = datasets.MNIST('data/', train=True, download=True, transform=dataset_to_transform['mnist']['train'])
test_ds = datasets.MNIST('data/', train=False, download=True, transform=dataset_to_transform['mnist']['val'])

from torch.utils.data import DataLoader
test_dl = DataLoader(test_ds, batch_size=32)

In [3]:
train_emnist = datasets.EMNIST('data/', split='balanced', train=True, download=True, transform=dataset_to_transform['mnist']['train'])
test_emnist = datasets.EMNIST('data/', split='balanced', train=False, download=True, transform=dataset_to_transform['mnist']['val'])
train_cifar = datasets.CIFAR10('data/', train=True, download=True, transform=dataset_to_transform['cifar10']['train'])
val_cifar = datasets.CIFAR10('data/', train=False, download=True, transform=dataset_to_transform['cifar10']['train'])

Files already downloaded and verified
Files already downloaded and verified


In [4]:
# client.py


class Client:
    """Base client.
    
    Parameters
    ----------
    client_id : str
        Id of the client.
    dataloader : DataLoader
        Local dataset used for training on the client.
    device : str, torch.device (default='cpu')
        Device type.
    """
    def __init__(self, client_id, dataloader, device='cpu'):
        self.client_id = client_id
        self.dataloader = dataloader
        self.device = device
        self._model = None
        self._device = None
        self._optimizer = None
        
        self._local_steps = 0
        
    @property
    def model(self):
        return self._model
    
    @model.setter
    def model(self, model):
        self._model = model
        self.device = next(self._model.parameters()).device

    @property
    def optimizer(self):
        return self._optimizer
    
    @optimizer.setter
    def optimizer(self, optimizer):
        self._optimizer = optimizer
    
    @property
    def device(self):
        return self._device

    @device.setter
    def device(self, device):
        self._device = device
    
    @property
    def local_steps(self):
        return self._local_steps
    
    @local_steps.setter
    def local_steps(self, v):
        self._local_steps = v
        
    def __len__(self):
        return len(self.dataloader.dataset)
    
    def update(self, criterion, num_epochs=1):
        """Algorithm 1 (ClientUpdate).
        
        Parameters
        ----------
        optim_cls : 
        optim_params :
        num_epochs (E) : int
            Number of epochs.
        criterion : 
        """
        self.model.train()
        self.model.to(self.device)
        self.local_steps = 0
        
        total_loss = np.zeros(num_epochs, dtype=np.float32)
        total_correct = np.zeros(num_epochs, dtype=np.float32)
        for i in range(num_epochs):
            for x, y in self.dataloader:
                x = x.to(self.device)
                y = y.to(self.device)
                
                self.optimizer.zero_grad()
                
                logits = self.model(x)
                loss = criterion(logits, y)
                loss.backward()
                self.optimizer.step()
                
                total_loss[i] += loss.item()
                total_correct[i] += (logits.argmax(-1) == y).sum().item()
                self.local_steps += 1
                
            if self.scheduler is not None:
                self.scheduler.step()
            
        self.model.to('cpu')
        results = {
            'loss': total_loss / len(self.dataloader),
            'accuracy': total_correct / len(self)
        }
        return results

    def get_gradients(self, criterion):
        self.model.eval()
        self.model.to(self.device)
        # clear gradients
        for p in self.model.parameters():
            if p.grad is not None and p.requires_grad:
                p.grad.zero_()
        for x, y in self.dataloader:
            x = x.to(self.device)
            y = y.to(self.device)

            logits = self.model(x)
            loss = criterion(logits, y)
            
            # accumulate the average gradient of each batch
            loss.backward()
        
        # normalize the accumulated gradient across batches
        grads = []
        for p in self.model.parameters():
            # what to do when the model has layers that don't require gradients?
            grads.append(p.grad / len(self.dataloader))
        
        return grads

In [5]:
# dataset.py
from torch.utils.data import Dataset, DataLoader


class SplitDataset(Dataset):
    """Dataset for a client partitioned by a list of indices."""
    def __init__(self, dataset, indices):
        self.dataset = dataset
        self.indices = list(indices)
        
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, index):
        return self.dataset[self.indices[index]]

In [6]:
# sampling.py
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader


def get_iid_shards(labels,
                   num_clients=100,
                   client_ids=None,
                   seed=None):
    """Returns a homogeneous (IID) mapping of client ID's to sample indices of a dataset.
    
    Parameters
    ----------
    labels : list, np.ndarray, torch.Tensor
        List of class labels for a dataset.
    num_clients : int (default=100)
        Number of clients used for training.
    client_ids : list, np.ndarray (default=None)
        List of client ID's. If `None`, defaults to a `range(num_clients)`.
    seed : int (default=None)
        Random state.
       
    Notes
    -----
    ```
    num_samples_per_client = len(labels) // num_clients # 60000 / 100 = 600 for MNIST
    ```
    """
    random_state = np.random.RandomState(seed)
    client_ids = client_ids or list(range(num_clients))
    
    # randomly shuffle sample indices to generate IID (homogeneous) clients
    indices_shuffled = random_state.choice(range(len(labels)), len(labels), replace=False)
    num_samples_per_client = len(indices_shuffled) // num_clients

    # assign `num_samples_per_client` random samples to each client
    for i, client_id in enumerate(client_ids):
        client_indices = indices_shuffled[i * num_samples_per_client : (i + 1) * num_samples_per_client]
        client_to_shard[client_id] = client_indices
    
    return client_to_shard
    
    
# def get_non_iid_shards(labels, 
#                        num_clients=100,
#                        client_ids=None,
#                        shard_size=300,
#                        drop_last=False,
#                        seed=None):
#     """Returns a heterogeneous (non-IID) mapping of client ID's to sample indices of a dataset.
    
#     Parameters
#     ----------
#     labels : list, np.ndarray, torch.Tensor
#         List of class labels for a dataset.
#     num_clients : int (default=100)
#         Number of clients used for training.
#     client_ids : list, np.ndarray (default=None)
#         List of client ID's. If `None`, defaults to a `range(num_clients)`.
#     shard_size : int (default=300)
#         Size of each shard to split labels by.
#     seed : int (default=None)
#         Random state.
       
#     Notes
#     -----
#     ```
#     num_shards = len(labels) // shard_size # 60000 / 300 = 200 for MNIST
#     num_shards_per_client = num_shards // num_clients # 200 / 100 = 2 for MNIST
#     ```
#     """
#     random_state = np.random.RandomState(seed)
#     client_ids = client_ids or list(range(num_clients))
#     classes = np.unique(labels)
#     num_classes = len(classes)

#     label_to_indices = {}
#     client_to_shard = {}
#     shards = []
#     shard_labels = []
#     # map each class to the sample indices in the dataset
#     for label in classes:
#         label_to_indices[label] = np.where(labels == label)[0]

#     # split each class labels indices to shards of size `shard_size`
#     for label in classes:
#         indices = label_to_indices[label]
#         num_extra = len(indices) % shard_size
#         # (num_shards_per_label, shard_size)
#         label_shards = indices[:-num_extra].reshape(-1, shard_size).tolist()

#         if num_extra > 0 and not drop_last:
#             label_shards.append(indices[-num_extra:])

#         # store the shards in a list to resample and assign to a client
#         shard_labels.extend([label] * len(label_shards))
#         shards.extend(label_shards)

#     num_shards = len(shard_labels) # number of shards of size `shard_size`
#     num_shards_per_client = num_shards // num_clients # how many shards fit in each client

#     # shuffle the shards and assign `num_shards_per_client` to each client
#     # each client should ideally have `num_shards_per_client` from a different class lablel
#     shard_indices_shuffled = random_state.choice(range(num_shards), 
#                                                  num_shards,
#                                                  replace=False)
#     for i, client_id in enumerate(client_ids):
#         client_to_shard[client_id] = []
#         shard_indices = shard_indices_shuffled[i * num_shards_per_client : (i + 1) * num_shards_per_client]
#         for idx in shard_indices:
#             client_to_shard[client_id].extend(shards[idx])
            
#     return client_to_shard


def get_client_shards(labels, 
                      is_iid=True,
                      num_clients=100,
                      client_ids=None,
                      shard_size=300,
                      seed=None):
    """Returns a heterogeneous (non-IID) mapping of client ID's to sample indices of a dataset.
    
    Parameters
    ----------
    labels : list, np.ndarray, torch.Tensor
        List of class labels for a dataset.
    num_clients : int (default=100)
        Number of clients used for training.
    client_ids : list, np.ndarray (default=None)
        List of client ID's. If `None`, defaults to a `range(num_clients)`.
    shard_size : int (default=300)
        Size of each shard to split labels by.
    seed : int (default=None)
        Random state.
       
    Notes
    -----
    ```
    num_shards = len(labels) // shard_size # 60000 / 300 = 200 for MNIST
    num_shards_per_client = num_shards // num_clients # 200 / 100 = 2 for MNIST
    ```
    """

    random_state = np.random.RandomState(seed)
    client_ids = client_ids or list(range(num_clients))
    client_to_shard = {c_id: [] for c_id in client_ids}
    if is_iid:
        # randomly shuffle sample indices if IID (homogoneous)
        sample_indices = random_state.choice(range(len(labels)), len(labels), replace=False)
    else:
        # sort sample indices by by label if non-IID (heterogeneous)
        sample_indices = np.argsort(labels).tolist()

    num_shards = len(labels) // shard_size
    num_shards_per_client = num_shards // num_clients # how many shards fit in each client
    shard_indices = set(range(num_shards))

    for i, client_id in enumerate(client_ids):
        client_shard_indices = random_state.choice(list(shard_indices), 
                                                   num_shards_per_client, 
                                                   replace=False)
        for shard_idx in client_shard_indices:
            client_to_shard[client_id].extend(
                sample_indices[shard_idx*shard_size : (shard_idx+1)*shard_size]
            )
            shard_indices.remove(shard_idx)
            
    return client_to_shard


def get_client_data(dataset, 
                    num_clients,
                    client_ids=None,
                    is_iid=True,
                    shard_size=300,
                    batch_size=32,
                    shuffle=True,
                    seed=None,
                    **kwargs):
    """Returns a mapping of client ID's to their corresponding train & validation dataloaders.
    
    Parameters
    ----------
    is_iid : bool (default=True)
        Boolean whether data is IID (Homogeneous) or non-IID (Heterogeneous)
    **kwargs
        Additional parameters used when instantiating each clients dataloader 
        
    """
    client_to_data = {}
    labels = dataset.targets
    client_to_shard = get_client_shards(
        labels,
        is_iid=is_iid,
        num_clients=num_clients,
        client_ids=client_ids,
        shard_size=shard_size,
        seed=seed
    )
    # iterate through each client and create train/val dataloaders using the shard indices
    for k in client_to_shard.keys():
        client_indices = client_to_shard[k]
        client_dataset = SplitDataset(dataset, client_indices)
        # reseed workers for reproducibility
#         g = torch.Generator()
#         g.manual_seed(0)
        client_to_data[k] = DataLoader(
            client_dataset, 
            batch_size=batch_size,
            shuffle=shuffle, 
#             worker_init_fn=seed_worker,
#             generator=g,
            **kwargs
        )
        
    return client_to_data


def get_clients(dataset,
                num_clients=100,
                client_ids=None,
                is_iid=True,
                shard_size=300,
                drop_last=True,
                batch_size=32,
                num_workers=0,
                device='cpu',
                **kwargs):
    client_to_data = get_client_data(
        dataset,
        num_clients=num_clients,
        client_ids=client_ids,
        is_iid=is_iid,
        shard_size=shard_size,
        drop_last=drop_last,
        batch_size=batch_size,
        num_workers=num_workers,
        **kwargs
    )
    clients = {
        k: Client(k, dl, device=device)
        for k, dl
        in client_to_data.items()
    }
    return clients

In [7]:
# model.py
import torch.nn as nn


class CNN(nn.Module):
    """CNN described in "Communication-Efficient Learning of Deep Networks
    from Decentralized Data" (https://arxiv.org/pdf/1602.05629.pdf).
    
    Parameters
    ----------
    in_features : int (default=1)
        Number of channels in the input image.
    num_classes : int (default=10)
        Number of class labels.
    """
    def __init__(self, in_features=1, num_classes=10):
        super(CNN, self).__init__()
        self._in_features = in_features
        self._num_classes = num_classes
        
        self.conv1 = nn.Conv2d(self._in_features,
                               32,
                               kernel_size=5)
        self.conv2 = nn.Conv2d(32,
                               64,
                               kernel_size=5)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, self._num_classes)
        
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=(2,2))
        
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.maxpool(x)
        x = self.relu(self.conv2(x))
        x = self.maxpool(x)
        x = x.reshape(len(x), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
    
class MLP(nn.Module):
    """CNN described in "Communication-Efficient Learning of Deep Networks
    from Decentralized Data" (https://arxiv.org/pdf/1602.05629.pdf).
    
    Parameters
    ----------
    in_features : int (default=784)
        Number input features.
    hidden_dim : int (cdefault=200)
        Number of hidden units.
    num_classes : int (default=10)
        Number of class labels.
    """
    def __init__(self, in_features=784, hidden_dim=200, num_classes=10):
        super(MLP, self).__init__()
        self._in_features = in_features
        self._hidden_dim = hidden_dim
        self._num_classes = num_classes
        
        self.fc1 = nn.Linear(self._in_features, self._hidden_dim)
        self.fc2 = nn.Linear(self._hidden_dim, self._num_classes)
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [8]:
# servers (aggregates parameters from local solvers)
    # base.py
    # fedavg.py
    # fedprox.py
    # feddane.py
    # fednova.py
    # fedopt.py
    # scaffold.py
# optimizers (local solvers)
    # fedprox.py
    # feddane.py
    # fedopt.py
    # scaffold.py
    # fednova.py
# utils
    # client.py
    # sampling.py
# models
    # mnist
        # cnn.py
        # mlp.py

In [110]:
# create a server for each algo
# create a client for each algo
# create an optimizer for each algo

In [111]:
# optimizers/fedprox.py
from torch.optim.optimizer import Optimizer


class FedProxSolver(Optimizer):
    """Implements FedProx local solver.
    
    This adds a proximal term to any clients optimizer.
    
    This wrapper allows us to pass in any torch.optim.Optimizer for
    a given client, not limited to SGD as originally proposed.
    
    Args:
        optimizer (torch.optim.Optimizer): local optimizer.
        mu (float): proximal term weight (default: 0)

    __ https://arxiv.org/pdf/1812.06127.pdf
        
    Example:
        >>> # train a model locally for a client
        >>> client_optimizer = torch.optim.SGD(model.parametesr(), lr=0.1)
        >>> client_optimizer = FedProxLocal(client_optimizer, mu=0.1)
        >>> client_optimizer.zero_grad()
        >>> loss_fn(model(input), target).backward()
        >>> client_optimizer.step()
    
    """
    
    def __init__(self, 
                 optimizer,
                 mu=0):
        if mu < 0.0:
            raise ValueError(f'Invalid mu value: {mu}')
        self.optimizer = optimizer
        self.mu = mu
        self.param_groups = self.optimizer.param_groups
        self.state = self.optimizer.state
#         self.state = defaultdict(dict)
    
    def _update(self, group):
        """Applies a proximal update to a parameter group."""
        for p in group['params']:
            if p.grad is None:
                continue
            state = self.state[p]
            p.data.add_(state['proximal'], alpha=-group['lr'])
        
    def step(self, closure=None):
        """Performs a single optimization step.
        
        Parameters
        ----------
        closure : bool
            A closure that reevaluates the model and returns the loss.
        """
        # set the initial (global) weights and proximal term before we update the client optimizer
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                state = self.state[p]
                if 'initial_weights' not in state:
                    state['initial_weights'] = torch.clone(p.data).detach()
                state['proximal'] = self.mu * (p.data - state['initial_weights'])

        loss = self.optimizer.step(closure=closure)
        for group in self.param_groups:
            self._update(group)
        return loss
    
    
class FedDaneSolver(Optimizer):
    """Implements FedDane local solver.
    
    Args:
        optimizer (torch.optim.Optimizer): local optimizer.
        mu (float): proximal term weight (default: 0)
    
        
    Example:
        >>> # train a model locally for a client
        >>> client_optimizer = torch.optim.SGD(model.parametesr(), lr=0.1)
        >>> client_optimizer = FedDaneLocal(client_optimizer, average_gradients, mu=0.1)
        >>> client_optimizer.zero_grad()
        >>> loss_fn(model(input), target).backward()
        >>> client_optimizer.step()
    
    __ https://arxiv.org/pdf/2001.01920.pdf
    """
    
    def __init__(self, 
                 optimizer,
                 average_gradients,
                 mu=0):
        if mu < 0.0:
            raise ValueError(f'Invalid mu value: {mu}')
        self.optimizer = optimizer
        self.average_gradients = average_gradients
        self.mu = mu
        self.param_groups = self.optimizer.param_groups
        self.state = self.optimizer.state
    
    def _update(self, group):
        """Applies a proximal update to a parameter group."""
        for p in group['params']:
            if p.grad is None:
                continue
            state = self.state[p]
            p.data.add_(state['proximal'] + state['grad_delta'], alpha=-group['lr'])
        
    def step(self, closure=None):
        """Performs a single optimization step.
        
        Parameters
        ----------
        closure : bool
            A closure that reevaluates the model and returns the loss.
        """
        # set the initial (global) weights and proximal term before we update the client optimizer
        for group in self.param_groups:
            for i, p in enumerate(group['params']):
                if p.grad is None:
                    continue
                state = self.state[p]
                if 'initial_weights' not in state:
                    state['initial_weights'] = torch.clone(p.data).detach()
                state['proximal'] = self.mu * (p.data - state['initial_weights'])
                if 'average_gradient' not in state:
                    state['average_gradient'] = torch.clone(self.average_gradients[i]).detach()
                state['grad_delta'] = state['average_gradient'] - p.grad.data

        loss = self.optimizer.step(closure=closure)
        for group in self.param_groups:
            self._update(group)
        return loss
    
    
class FedNovaSolver(Optimizer):
    """Implements FedNova local solver.
    
    Args:
        optimizer (torch.optim.Optimizer): local optimizer.
        mu (float): proximal term weight (default: 0)

    __ https://arxiv.org/pdf/2007.07481.pdf
        
    Example:
        >>> # train a model locally for a client
        >>> client_optimizer = torch.optim.SGD(model.parametesr(), lr=0.1)
        >>> client_optimizer = FedNovaLocal(client_optimizer, mu=0.1)
        >>> client_optimizer.zero_grad()
        >>> loss_fn(model(input), target).backward()
        >>> client_optimizer.step()
    """
    
    def __init__(self, 
                 optimizer,
                 mu=0):
        if mu < 0.0:
            raise ValueError(f'Invalid mu value: {mu}')
        self.optimizer = optimizer
        self.mu = mu
        self.param_groups = self.optimizer.param_groups
        self.state = self.optimizer.state
#         self.state = defaultdict(dict)
    
    def _update(self, group):
        """Applies a proximal update to a parameter group."""
        for p in group['params']:
            if p.grad is None:
                continue
            state = self.state[p]
            p.data.add_(state['proximal'], alpha=-group['lr'])
        
    def step(self, closure=None):
        """Performs a single optimization step.
        
        Parameters
        ----------
        closure : bool
            A closure that reevaluates the model and returns the loss.
        """
        # set the initial (global) weights and proximal term before we update the client optimizer
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                state = self.state[p]
                if 'initial_weights' not in state:
                    state['initial_weights'] = torch.clone(p.data).detach()
                state['proximal'] = self.mu * (p.data - state['initial_weights'])

        # update the weights and gradient with the client optimizer
        loss = self.optimizer.step(closure=closure)
        
        # update the weights by adding the (negative) proximal term
        for group in self.param_groups:
            self._update(group)
        
        # accumualte gradients after calculating loss
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue
                
                momentum = group.get('momentum', 0)
                
                state = self.state[p]
                if 'local_step' not in state:
                    state['local_step'] = 0
                state['local_step'] += 1
                
                # momentum (1 - p^t) / (1 - p)
                a = (1 - momentum ** state['local_step']) / (1 - momentum)
                # proximal (1 - lr * mu)^t
                a *= (1 - group['lr'] * self.mu) ** (state['local_step']-1)
                # record the norm factor (a) to divide the l1-norm during aggregation
                if 'norm_factor' not in state:
                    state['norm_factor'] = []
                state['norm_factor'].append(a)
                
                if 'cgrad' not in state:
                    state['cgrad'] = torch.clone(p.grad.data).detach()
                    state['cgrad'].mul_(group['lr']) # do we need the lr ?
                    state['cgrad'].mul_(a) # G * a
                else:
                    state['cgrad'].add_(p.grad.data, alpha=group['lr'])
                    state['cgrad'].mul_(a) # G * a
                    
        return loss

In [112]:
# servers/base.py
from copy import deepcopy
from collections import defaultdict
from torch.utils.tensorboard.writer import SummaryWriter


class BaseFederater:
    """Base Federater.
    
    Parameters
    ----------
    model : nn.Module
    dataset : torch.utils.data.Dataset
    num_clients (K) : int (default=100)
        Number of clients to partition `dataset`.
    batch_size (B) : int, dict[str, int] (defualt=32)
        Number of samples per batch to load on each client. 
        Can be a dictionary mapping each client ID to it's corresponding batch size
        to allow for various batch sizes across clients.
    shard_size : int (default=300)
    is_iid : bool (default=False)
    drop_last : bool (default=True)
    num_workers : int (default=0)
    device : str (default='cpu')
    """
    def __init__(self, 
                 model,
                 clients,
                 client_optimizer_cls,
                 client_optimizer_params,
                 server_optimizer=None,
                 client_scheduler_cls=None,
                 client_scheduler_params=None,
                 server_scheduler=None,
                 C=0.1,
                 seed=None,
                 writer=None):
        self.model = model
        self.clients = clients
        self.client_optimizer_cls = client_optimizer_cls
        self.client_optimizer_params = client_optimizer_params
        self.server_optimizer = server_optimizer
        self.client_scheduler_cls = client_scheduler_cls
        self.client_scheduler_params = client_scheduler_params
        self.server_scheduler = server_scheduler
        self.C = C
        self.writer = writer or SummaryWriter()
        
        self.client_ids = list(self.clients.keys())
        self.num_clients = len(self.clients)
        self.num_samples = sum([len(c) for c in self.clients.values()]) # n
        self.client_weights = [len(c) / self.num_samples for c in self.clients.values()]
        
        self.device = next(self.model.parameters()).device
        self._global_round = 0
        self._random_state = np.random.RandomState(seed)
        
    @property
    def global_round(self):
        return self._global_round

    @global_round.setter
    def global_round(self, global_round):
        self._global_round = global_round
    
    def aggregate(self):
        raise NotImplementedError
        
    def update(self, 
               client_ids, 
               criterion,
               num_epochs, 
               straggler_rate=0):
        """Performs a full communication round.
        
        Parameters
        ----------
        client_ids (S_t): list, np.ndarray
            List of client ID's to train.
        criterion : nn.Module
            Loss function to optimize on each client.
        num_epochs (E): int
            Number of epochs to train on each client.
            
        Returns
        -------
        metrics_dict : dict
            Dictionary mapping each metric to the average score across `client_ids`
        """
        # send the global model parameters to each client
        self.send_model()
        
        metrics_dict = defaultdict(lambda: 0)
        for k in client_ids:
            # instantiate client optimizer and scheduler
            client = self.clients[k]
            client.optimizer = self.get_client_optimizer(client)
            client.scheduler = self.get_client_scheduler(client.optimizer)
            
            # for heterogeneity experiments we can train clients for varying epochs (stragglers)
            if self._random_state.random() < straggler_rate:
                client_epochs = self._random_state.choice(range(1, num_epochs+1))
            else:
                client_epochs = num_epochs
                
            # update the client weights and record the local training metrics
            client_metrics_dict = client.update(
                criterion,
                num_epochs=client_epochs,
            )
            
            # update the summary writer and record loss/acc from the client
            for metric, values in client_metrics_dict.items():
                self.writer.add_scalar(f'client/{k}/{metric}', values[-1], self.global_round)
                metrics_dict[metric] += values[-1] / len(client_ids)
        
        # aggregate the parameters of the local solvers
        self.aggregate()
        if self.server_scheduler is not None:
            self.server_scheduler.step()
        
        return metrics_dict
        
    def fit(self, 
            num_rounds,
            criterion, 
            num_epochs,
            val_dl=None,
            straggler_rate=0,
            eval_every_n=1):
        # subset a sample of `m` clients each round
        m = max(int(np.ceil(self.num_clients * self.C)), 1)
        for t in range(num_rounds):
            self.global_round += 1
            
            # update a subset of clients with the local solver
            S = self._random_state.choice(self.client_ids, m, replace=False)
            train_metrics = self.update(client_ids=S, 
                                        criterion=criterion,
                                        num_epochs=num_epochs, 
                                        straggler_rate=straggler_rate)
            
            if eval_every_n is not None and t % eval_every_n == 0 and val_dl is not None:
                template_str = f'round {self.global_round}'
                val_metrics = self.validate(val_dl, criterion)
                for metric, value in train_metrics.items():
                    self.writer.add_scalar(f'train/{metric}', value, self.global_round)
                    template_str += f' - train_{metric} : {value:0.4f}'
                for metric, value in val_metrics.items():
                    self.writer.add_scalar(f'val/{metric}', value, self.global_round)
                    template_str += f' - val_{metric} : {value:0.4f}'
                
                print(template_str)
    
    def get_client_optimizer(self, client):
        """Returns a client optimizer (local solver).
        
        Parameters
        ----------
        params : iterable
            Client parameters to optimize
        optimizer_params : dict
            Client optimizer hyperparameters
        
        Returns
        -------
        torch.optim.Optimizer
        """
        optimizer_params = self.client_optimizer_params or {}
        return self.client_optimizer_cls(client.model.parameters(), **optimizer_params)
    
    def get_client_scheduler(self, optimizer):
        """Returns a LR scheduler for a client optimizer.
        
        Parameters
        ----------
        optimizer : torch.optim.Optimizer
            Client optimizer
        
        Returns
        -------
        torch.optim.lr_scheduler._LRScheduler or None
            Client LR scheduler, or None if not specified
        """
        if self.client_scheduler_cls is not None:
            scheduler_params = self.client_scheduler_params or {}
            return self.client_scheduler_cls(optimizer, **scheduler_params)
        else:
            return None
        
    def validate(self, val_dl, criterion):
#         self.model.to(self.device)
        self.model.eval()
        loss = 0 
        correct = 0
        with torch.no_grad():
            for x, y in val_dl:
                x = x.to(self.device)
                y = y.to(self.device)
                logits = self.model(x)
                correct += (logits.argmax(-1) == y).sum().item()
                loss += criterion(logits, y).item()
                
        results = {
            'loss': loss / len(val_dl),
            'accuracy': correct / len(val_dl.dataset)
        }
        return results
    
    def send_model(self, client_ids=None):
        """Send the current state of the global model to each client."""
        if client_ids is None:
            client_ids = self.client_ids
        for client_id in client_ids:
            self.clients[client_id].model = deepcopy(self.model)
            
    def get_gradients(self, client_ids, criterion):
        self.send_model(client_ids)
        grads = []
        for k, client_id in enumerate(client_ids):
            client = self.clients[client_id]
            client_grads = client.get_gradients(criterion)
            grads.append(client_grads)
        return grads

In [113]:
class FedAvg(BaseFederater):
    
    def __init__(self, 
                 model,
                 clients,
                 client_optimizer_cls,
                 client_optimizer_params,
                 client_scheduler_cls=None,
                 client_scheduler_params=None,
                 C=0.1,
                 seed=None,
                 writer=None):
        super().__init__(model,
                         clients,
                         client_optimizer_cls=client_optimizer_cls,
                         client_optimizer_params=client_optimizer_params,
                         client_scheduler_cls=client_scheduler_cls,
                         client_scheduler_params=client_scheduler_params,
                         C=C,
                         seed=seed,
                         writer=writer)
    
    def aggregate(self):
        global_state = {} # self.model.state_dict()
        for k, (client_id, client) in enumerate(self.clients.items()):
            local_state = client.model.state_dict()
            for layer_name, param in local_state.items():
                if k == 0:
                    global_state[layer_name] = self.client_weights[k] * param
                else:
                    global_state[layer_name] += self.client_weights[k] * param

        self.model.load_state_dict(global_state)
                    

class FedProx(BaseFederater):
    def __init__(self, 
                 model,
                 clients,
                 client_optimizer_cls,
                 client_optimizer_params,
                 server_optimizer,
                 mu=0,
                 client_scheduler_cls=None,
                 client_scheduler_params=None,
                 server_scheduler=None,
                 C=0.1,
                 seed=None,
                 writer=None):
        super().__init__(model,
                         clients,
                         client_optimizer_cls,
                         client_optimizer_params,
                         server_optimizer=server_optimizer,
                         server_scheduler=server_scheduler,
                         client_scheduler_cls=client_scheduler_cls,
                         client_scheduler_params=client_scheduler_params,
                         C=C,
                         seed=seed,
                         writer=writer)
        self.mu = mu
            
    def get_client_optimizer(self, client):
        optimizer_params = self.client_optimizer_params or {}
        client_optimizer = self.client_optimizer_cls(client.model.parameters(), **optimizer_params)
        return FedProxSolver(client_optimizer, mu=self.mu)
    
    def aggregate(self):
        self.server_optimizer.zero_grad()
        for k, client in enumerate(self.clients.values()):
            for p_server, p_client in zip(self.model.parameters(), client.model.parameters()):
                if p_server.requires_grad:
                    if k == 0:
                        p_server.grad = self.client_weights[k] * (p_server.data - p_client.data)
                    else:
                        p_server.grad.data.add_(p_server.data - p_client.data, alpha=self.client_weights[k])
        
        self.server_optimizer.step()

    
class FedOpt(BaseFederater):
    
    def __init__(self,
                 model,
                 clients,
                 client_optimizer_cls,
                 client_optimizer_params,
                 server_optimizer,
                 server_scheduler=None,
                 client_scheduler_cls=None,
                 client_scheduler_params=None,
                 C=0.1,
                 seed=None,
                 writer=None):
        super().__init__(model,
                         clients,
                         client_optimizer_cls,
                         client_optimizer_params,
                         server_optimizer=server_optimizer,
                         server_scheduler=server_scheduler,
                         client_scheduler_cls=client_scheduler_cls,
                         client_scheduler_params=client_scheduler_params,
                         C=C,
                         seed=seed,
                         writer=writer)
    
    def aggregate(self):
        """
        
        """
        self.server_optimizer.zero_grad()
        # iterate through each client
        for k, client in enumerate(self.clients.values()):
            for p_server, p_client in zip(self.model.parameters(), client.model.parameters()):
                if p_server.requires_grad:
                    if k == 0:
                        p_server.grad = client_weights[k] * (p_server.data - p_client.data)
                    else:
                        p_server.grad.add_(p_server.data - p_client.data, alpha=self.client_weights[k])
        
        self.server_optimizer.step()
        
        
class FedNova(BaseFederater):
    """FedNova
    
    https://arxiv.org/pdf/2007.07481.pdf
    """
    def __init__(self,
                 model,
                 clients,
                 server_optimizer,
                 client_optimizer_cls,
                 client_optimizer_params,
                 mu=0,
                 server_scheduler=None,
                 client_scheduler_cls=None,
                 client_scheduler_params=None,
                 C=0.1,
                 seed=None,
                 writer=None):
        super().__init__(model,
                         clients,
                         client_optimizer_cls=client_optimizer_cls,
                         client_optimizer_params=client_optimizer_params,
                         server_optimizer=server_optimizer,
                         server_scheduler=server_scheduler,
                         client_scheduler_cls=client_scheduler_cls,
                         client_scheduler_params=client_scheduler_params,
                         C=C,
                         seed=seed,
                         writer=writer)
        self.mu = mu
        
    def get_client_optimizer(self, client):
        optimizer_params = self.client_optimizer_params or {}
        client_optimizer = self.client_optimizer_cls(client.model.parameters(), **optimizer_params)
        client_optimizer = FedNovaSolver(client_optimizer, mu=self.mu)
        return client_optimizer
    
    def aggregate(self):
        """ """
        
        self.server_optimizer.zero_grad()
        # iterate through each client and set gradients
        for k, client in enumerate(self.clients.values()):
            # skip clients with no optimizer
            # we may want to use the weights of the local model instead
            if client.optimizer is None:
                continue
            for group_server, group_client in zip(self.server_optimizer.param_groups, 
                                                  client.optimizer.param_groups):
                for p_server, p_client in zip(group_server['params'], group_client['params']):
                    if p_server.requires_grad:
                        state = client.optimizer.state[p_client]
                        w = self.client_weights[k]
                        G_a = state['cgrad']
                        a = torch.tensor(state['norm_factor'])
                        d = G_a / a.abs().sum()
                        tau_eff = client.local_steps
                        if p_server.grad is None:
                            p_server.grad = tau_eff * w * d  # need to take lr off of G ? jk lr is necessary for client (local)
                        else:
                            p_server.grad.add_(d, alpha=tau_eff * w)

        self.server_optimizer.step()
        
        
class FedDane(BaseFederater):
    def __init__(self, 
                 model,
                 clients,
                 client_optimizer_cls,
                 client_optimizer_params,
                 server_optimizer,
                 mu=0,
                 client_scheduler_cls=None,
                 client_scheduler_params=None,
                 server_scheduler=None,
                 C=0.1,
                 seed=None,
                 writer=None):
        super().__init__(model,
                         clients,
                         client_optimizer_cls,
                         client_optimizer_params,
                         server_optimizer=server_optimizer,
                         server_scheduler=server_scheduler,
                         client_scheduler_cls=client_scheduler_cls,
                         client_scheduler_params=client_scheduler_params,
                         C=C,
                         seed=seed,
                         writer=writer)
        self.mu = mu
        self.average_gradients = None
            
    def get_client_optimizer(self, client):
        optimizer_params = self.client_optimizer_params or {}
        client_optimizer = self.client_optimizer_cls(client.model.parameters(), **optimizer_params)
        client_optimizer = FedDaneSolver(client_optimizer, 
                                         average_gradients=self.average_gradients,
                                         mu=self.mu)
        return client_optimizer
    
    def fit(self, 
            num_rounds,
            criterion, 
            num_epochs,
            val_dl=None,
            straggler_rate=0,
            eval_every_n=1):
        # subset a sample of `m` clients each round
        m = max(int(np.ceil(self.num_clients * self.C)), 1)
        
        for t in range(num_rounds):
            self.global_round += 1
            
            # calculate the average gradient on a subset of clients
            S_grad = self._random_state.choice(self.client_ids, m, replace=False)
            self.set_average_gradients(S_grad, criterion)
            
            # update a subset of clients with the local solver
            S = self._random_state.choice(self.client_ids, m, replace=False)
            train_metrics = self.update(client_ids=S, 
                                        criterion=criterion,
                                        num_epochs=num_epochs, 
                                        straggler_rate=straggler_rate)
            
            if eval_every_n is not None and t % eval_every_n == 0 and val_dl is not None:
                template_str = f'round {self.global_round}'
                val_metrics = self.validate(val_dl, criterion)
                for metric, value in train_metrics.items():
                    self.writer.add_scalar(f'train/{metric}', value, self.global_round)
                    template_str += f' - train_{metric} : {value:0.4f}'
                for metric, value in val_metrics.items():
                    self.writer.add_scalar(f'val/{metric}', value, self.global_round)
                    template_str += f' - val_{metric} : {value:0.4f}'
                
                print(template_str)
    
    def aggregate(self):
        self.server_optimizer.zero_grad()
        for k, client in enumerate(self.clients.values()):
            for p_server, p_client in zip(self.model.parameters(), client.model.parameters()):
                if p_server.requires_grad:
                    if k == 0:
                        p_server.grad = self.client_weights[k] * (p_server.data - p_client.data)
                    else:
                        p_server.grad.add_(p_server.data - p_client.data, alpha=self.client_weights[k])
        
        self.server_optimizer.step()
        
    def set_average_gradients(self, client_ids, criterion):
        grads = self.get_gradients(client_ids, criterion)
        average_gradients = [0] * len(grads[0])
        for client_grads in grads:
            for i, g in enumerate(client_grads):
                average_gradients[i] += g
        self.average_gradients = [g / len(grads) for g in average_gradients]

In [116]:
class Config:
    
    fed_avg = {
        'mnist': {
            'clients': {
                'num_clients': 100,
                'shard_size': 300,
                'batch_size': 50,
                'is_iid': False,
            },
            'client_optimizer': 'SGD',
            'client_optimizer_params': {
                'lr': 0.1,
            },
            'federater': {
                'C': 0.1
            },
            'fit': {
                'num_rounds': 3,
                'num_epochs': 5
            }
        }
    }
    fed_prox = {
        'optimizer': 'SGD',
        'params': {
            'lr': 1,
            'momentum': 0.1
        }
    }

config = Config()

In [117]:
# Fed avg baselien
seed = 42069
device = 'cpu'
num_workers = 0
experiment_name = 'fedavg_baseline'
set_state(seed)
writer = SummaryWriter(os.path.join('logs', experiment_name))

client_params = config.fed_avg['mnist']['clients']
clients = get_clients(
    train_ds, 
    num_workers=num_workers,
    seed=seed,
    device=device,
    **client_params,
)

model = CNN()
# client optimizer
# client_optimizer_cls = torch.optim.SGD
# client_optimizer_params = {
#     'lr': 0.1,
# }
client_optimizer_cls = getattr(torch.optim, config.fed_avg['mnist']['client_optimizer'])
client_optimizer_params = config.fed_avg['mnist']['client_optimizer_params']
criterion = nn.CrossEntropyLoss()

fed_params = config.fed_avg['mnist']['federater']
fed_params['seed'] = seed
# fed_params = {
#     'seed': seed,
#     'C': 0.1,
# }
num_rounds = config.fed_avg['mnist']['fit']['num_rounds']
num_epochs = config.fed_avg['mnist']['fit']['num_epochs']

federater_prox = FedAvg(model,
                    clients=clients,
                    client_optimizer_cls=client_optimizer_cls,
                    client_optimizer_params=client_optimizer_params,
                    **fed_params)
federater_prox.fit(num_rounds=num_rounds, criterion=criterion, num_epochs=num_epochs, val_dl=test_dl)

round 1 - train_loss : 0.0522 - train_accuracy : 0.9828 - val_loss : 2.2997 - val_accuracy : 0.1437
round 2 - train_loss : 0.0411 - train_accuracy : 0.9868 - val_loss : 2.3005 - val_accuracy : 0.1852
round 3 - train_loss : 0.0404 - train_accuracy : 0.9873 - val_loss : 2.2985 - val_accuracy : 0.1398


In [133]:
%load_ext tensorboard

%tensorboard --bind_all --logdir logs/fedavg_avg --host localhost

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
TensorFlow installation not found - running with reduced feature set.
Error: Must not specify both --host and --bind_all.

In [119]:
os.listdir('logs/fedavg_baseline/')

['events.out.tfevents.1624399407.64acb651e23a.401.20',
 'events.out.tfevents.1624399370.64acb651e23a.401.19',
 'events.out.tfevents.1624399485.64acb651e23a.401.27',
 'events.out.tfevents.1624399462.64acb651e23a.401.25',
 'events.out.tfevents.1624399450.64acb651e23a.401.23',
 'events.out.tfevents.1624399444.64acb651e23a.401.22',
 'events.out.tfevents.1624399432.64acb651e23a.401.21']

In [85]:
# Fed avg baselien
seed = 42069
device = 'cpu'
set_state(seed)
model_avg = CNN()
clients_avg = get_clients(
    train_ds, 
    num_clients=100,
    is_iid=False,
    shard_size=300,
    batch_size=10,
    num_workers=0,
    seed=seed,
    device=device
)
# server optimizer
server_optimizer_params = {
    'lr': 1,
    'momentum': 0
}
server_optimizer = torch.optim.SGD(model_avg.parameters(), **server_optimizer_params)

# client optimizer
client_optimizer_cls = torch.optim.SGD
client_optimizer_params = {
    'lr': 0.1,
}
criterion = nn.CrossEntropyLoss()

fed_params = {
    'seed': seed,
    'C': 0.1,
}

federater_avg = FedAvg(model_avg,
                    clients=clients_avg,
                    client_optimizer_cls=client_optimizer_cls,
                    client_optimizer_params=client_optimizer_params,
                    **fed_params)
federater_avg.fit(num_rounds=5, criterion=criterion, num_epochs=1, val_dl=test_dl)

[12 56 43 34 32 55 21  2 41 77]
round 1 - train_loss : 0.6275 - train_accuracy : 0.8185 - val_loss : 2.2982 - val_accuracy : 0.1241
[ 1 10 86 59 64 47  3 27 80 72]
round 2 - train_loss : 0.7072 - train_accuracy : 0.7943 - val_loss : 2.2948 - val_accuracy : 0.1811
[91 80  2 34 63 97 12 14 49 89]
round 3 - train_loss : 0.6144 - train_accuracy : 0.8287 - val_loss : 2.2911 - val_accuracy : 0.2160
[ 5 14 29 47 48  3 74 62 64 66]
round 4 - train_loss : 0.6083 - train_accuracy : 0.8285 - val_loss : 2.2900 - val_accuracy : 0.2767
[73 94 87 63  3 44 23 88 59 84]
round 5 - train_loss : 0.5569 - train_accuracy : 0.8543 - val_loss : 2.2860 - val_accuracy : 0.3574


In [86]:
# Fed avg baselien
seed = 42069
device = 'cpu'
num_workers = 0
experiment_name = 'fedavg_baseline'
set_state(seed)
writer = SummaryWriter(os.path.join('logs', experiment_name))

data_params = config.fed_avg['mnist']['data']
model_prox = CNN()
clients_prox = get_clients(
    train_ds, 
    num_workers=num_workers,
    seed=seed,
    device=device
    **data_params,
)
# server optimizer
server_optimizer_params = {
    'lr': 1,
    'momentum': 0
}
server_optimizer = torch.optim.SGD(model_prox.parameters(), **server_optimizer_params)

# client optimizer
client_optimizer_cls = torch.optim.SGD
client_optimizer_params = {
    'lr': 0.1,
}
criterion = nn.CrossEntropyLoss()

fed_params = {
    'seed': seed,
    'mu': 0,
    'C': 0.1,
}

federater_prox = FedProx(model_prox,
                    clients=clients_prox,
                    client_optimizer_cls=client_optimizer_cls,
                    client_optimizer_params=client_optimizer_params,
                    server_optimizer=server_optimizer,
                    **fed_params)
federater_prox.fit(num_rounds=100, criterion=criterion, num_epochs=1, val_dl=test_dl)

[12 56 43 34 32 55 21  2 41 77]
round 1 - train_loss : 0.6275 - train_accuracy : 0.8185 - val_loss : 2.2982 - val_accuracy : 0.1241
[ 1 10 86 59 64 47  3 27 80 72]
round 2 - train_loss : 0.7070 - train_accuracy : 0.7947 - val_loss : 2.2948 - val_accuracy : 0.1812
[91 80  2 34 63 97 12 14 49 89]
round 3 - train_loss : 0.6384 - train_accuracy : 0.8200 - val_loss : 2.2918 - val_accuracy : 0.2129
[ 5 14 29 47 48  3 74 62 64 66]
round 4 - train_loss : 0.6047 - train_accuracy : 0.8390 - val_loss : 2.2907 - val_accuracy : 0.2724
[73 94 87 63  3 44 23 88 59 84]
round 5 - train_loss : 0.5962 - train_accuracy : 0.8512 - val_loss : 2.2884 - val_accuracy : 0.3317


In [46]:
list(model_avg.parameters())[0][0]

tensor([[[ 0.1237,  0.0496, -0.0533, -0.0223,  0.0750],
         [ 0.1293,  0.0621, -0.1308,  0.1752, -0.0304],
         [ 0.1696,  0.1714, -0.1323, -0.0063,  0.0882],
         [-0.0299,  0.1077,  0.0123,  0.1446,  0.1252],
         [ 0.1886,  0.1777, -0.1097, -0.0612, -0.1472]]],
       grad_fn=<SelectBackward>)

In [43]:
list(model_avg.parameters())[0][0]

tensor([[[ 0.1241,  0.0502, -0.0526, -0.0217,  0.0757],
         [ 0.1296,  0.0627, -0.1302,  0.1758, -0.0299],
         [ 0.1698,  0.1716, -0.1321, -0.0061,  0.0883],
         [-0.0298,  0.1078,  0.0124,  0.1448,  0.1253],
         [ 0.1890,  0.1780, -0.1095, -0.0609, -0.1471]]],
       grad_fn=<SelectBackward>)

In [56]:
new_avg = torch.stack([federater_avg.client_weights[i] * list(clients_avg[i].model.parameters())[0][0] for i in range(100)]).sum(0)
new_avg

tensor([[[ 0.1237,  0.0496, -0.0533, -0.0223,  0.0750],
         [ 0.1293,  0.0621, -0.1308,  0.1752, -0.0304],
         [ 0.1696,  0.1714, -0.1323, -0.0063,  0.0882],
         [-0.0299,  0.1077,  0.0123,  0.1446,  0.1252],
         [ 0.1886,  0.1777, -0.1097, -0.0612, -0.1472]]],
       grad_fn=<SumBackward1>)

In [64]:
(list(model_avg.parameters())[0][0] - new_avg)[0][0][0].item()

0.0003355368971824646

In [51]:
g = []
for i in range(100):
    g.append(list(model_avg.parameters())[0][0] - list(clients_avg[i].model.parameters())[0][0])

In [68]:
g[12][0][0][0].item() * federater_avg.client_weights[12]

0.00033553242683410646

In [764]:
list(model_avg.parameters())[0][0] - list(clients_avg[0].model.parameters())[0][0]

tensor([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]], grad_fn=<SubBackward0>)

In [87]:
list(model_avg.parameters())[0][0] - list(model_prox.parameters())[0][0]

tensor([[[-6.2098e-04, -8.3167e-04, -8.5653e-04, -1.0109e-04, -1.4994e-05],
         [-1.3470e-03, -1.2671e-03, -7.1046e-04,  1.7881e-05, -3.6251e-04],
         [-1.1452e-03, -1.8276e-03, -1.0516e-03, -6.7298e-04, -6.9550e-04],
         [-1.1875e-03, -1.6784e-03, -7.8158e-04, -3.3040e-04, -7.4774e-05],
         [-1.0530e-03, -9.4239e-04, -5.7211e-04, -4.1787e-04,  1.0461e-04]]],
       grad_fn=<SubBackward0>)

In [89]:
# Fed avg baselien
seed = 42069
device = 'cpu'
set_state(seed)
model_nova = CNN()
clients_nova = get_clients(
    train_ds, 
    num_clients=100,
    is_iid=False,
    shard_size=300,
    batch_size=10,
    num_workers=0,
    seed=seed,
    device=device
)
# server optimizer
server_optimizer_params = {
    'lr': 1,
    'momentum': 0.9
}
server_optimizer = torch.optim.SGD(model_nova.parameters(), **server_optimizer_params)

# client optimizer
client_optimizer_cls = torch.optim.SGD
client_optimizer_params = {
    'lr': 0.1,
}
criterion = nn.CrossEntropyLoss()

fed_params = {
    'seed': seed,
    'C': 0.1,
    'mu': 0.1,
}

federater_nova = FedNova(model_nova,
                    clients=clients_nova,
                    client_optimizer_cls=client_optimizer_cls,
                    client_optimizer_params=client_optimizer_params,
                         server_optimizer=server_optimizer,
                    **fed_params)
federater_nova.fit(num_rounds=5, criterion=criterion, num_epochs=1, val_dl=test_dl)

[12 56 43 34 32 55 21  2 41 77]
round 1 - train_loss : 0.7237 - train_accuracy : 0.7873 - val_loss : 2.3106 - val_accuracy : 0.1532
[ 1 10 86 59 64 47  3 27 80 72]
round 2 - train_loss : 0.8594 - train_accuracy : 0.7323 - val_loss : 2.3106 - val_accuracy : 0.1536
[91 80  2 34 63 97 12 14 49 89]
round 3 - train_loss : 0.7307 - train_accuracy : 0.7828 - val_loss : 2.3104 - val_accuracy : 0.1576
[ 5 14 29 47 48  3 74 62 64 66]
round 4 - train_loss : 0.7079 - train_accuracy : 0.8025 - val_loss : 2.3098 - val_accuracy : 0.1640
[73 94 87 63  3 44 23 88 59 84]
round 5 - train_loss : 0.8606 - train_accuracy : 0.7702 - val_loss : 2.3088 - val_accuracy : 0.1728


In [498]:
seed = 42069
device = 'cpu'
set_state(random_state)
model = CNN()
clients = get_clients(
    train_ds, 
    num_clients=100,
    is_iid=False,
    shard_size=300,
    batch_size=10,
    num_workers=0,
    seed=seed,
    device=device
)
# server optimizer
server_optimizer_params = {
    'lr': 1,
    'momentum': 0
}
server_optimizer = torch.optim.SGD(model.parameters(), **server_optimizer_params)

# client optimizer
client_optimizer_cls = torch.optim.SGD
client_optimizer_params = {
    'lr': 0.1,
}
criterion = nn.CrossEntropyLoss()

fed_params = {
    'mu': 0,
    'seed': seed,
}

federater = FedNova(model,
                    clients=clients,
                    client_optimizer_cls=client_optimizer_cls,
                    client_optimizer_params=client_optimizer_params,
                    server_optimizer=server_optimizer,
                    **fed_params)

# federater.fit(num_rounds=1, criterion=criterion, num_epochs=2, val_dl=test_dl)
federater.fit(num_rounds=5, criterion=criterion, num_epochs=1, val_dl=test_dl)

round 1 - train_loss : 0.6275 - train_accuracy : 0.8185 - val_loss : 2.2982 - val_accuracy : 0.1241
round 2 - train_loss : 0.7086 - train_accuracy : 0.7940 - val_loss : 2.2951 - val_accuracy : 0.2126
round 3 - train_loss : 0.6189 - train_accuracy : 0.8248 - val_loss : 2.2946 - val_accuracy : 0.2199
round 4 - train_loss : 0.4919 - train_accuracy : 0.8758 - val_loss : 2.2911 - val_accuracy : 0.2282
round 5 - train_loss : 0.5117 - train_accuracy : 0.8835 - val_loss : 2.2805 - val_accuracy : 0.2499


In [ ]:
federater.fit(num_rounds=5, criterion=criterion, num_epochs=1, val_dl=test_dl)

In [ ]:
round 1 - train_loss : 0.0692 - train_accuracy : 0.9733 - val_loss : 2.2971 - val_accuracy : 0.1301

In [63]:
federater.fit(num_rounds=100, criterion=criterion, num_epochs=20, val_dl=test_dl)

round 1 - train_loss : 0.0896 - train_accuracy : 0.9767 - val_loss : 2.3033 - val_accuracy : 0.1749
round 2 - train_loss : 0.0930 - train_accuracy : 0.9748 - val_loss : 2.2953 - val_accuracy : 0.1983
round 3 - train_loss : 0.1644 - train_accuracy : 0.9507 - val_loss : 2.2892 - val_accuracy : 0.1776
round 4 - train_loss : 0.0562 - train_accuracy : 0.9815 - val_loss : 2.2814 - val_accuracy : 0.1884
round 5 - train_loss : 0.1558 - train_accuracy : 0.9643 - val_loss : 2.2728 - val_accuracy : 0.1976
round 6 - train_loss : 0.1310 - train_accuracy : 0.9698 - val_loss : 2.2597 - val_accuracy : 0.2462
round 7 - train_loss : 0.1211 - train_accuracy : 0.9558 - val_loss : 2.2503 - val_accuracy : 0.3114
round 8 - train_loss : 0.1018 - train_accuracy : 0.9707 - val_loss : 2.2407 - val_accuracy : 0.3875
round 9 - train_loss : 0.1268 - train_accuracy : 0.9698 - val_loss : 2.2280 - val_accuracy : 0.4173
round 10 - train_loss : 0.0696 - train_accuracy : 0.9813 - val_loss : 2.2142 - val_accuracy : 0.4048

round 83 - train_loss : 0.0351 - train_accuracy : 0.9910 - val_loss : 0.4455 - val_accuracy : 0.9396


KeyboardInterrupt: 

In [2072]:
federater.fit(num_rounds=40, criterion=criterion, num_epochs=5, val_dl=test_dl)

round 11 - train_loss : 0.0856 - train_accuracy : 0.9775 - val_loss : 0.3990 - val_accuracy : 0.9443
round 12 - train_loss : 0.0996 - train_accuracy : 0.9710 - val_loss : 0.3361 - val_accuracy : 0.9469
round 13 - train_loss : 0.0887 - train_accuracy : 0.9742 - val_loss : 0.2864 - val_accuracy : 0.9494
round 14 - train_loss : 0.0779 - train_accuracy : 0.9803 - val_loss : 0.2479 - val_accuracy : 0.9513
round 15 - train_loss : 0.0763 - train_accuracy : 0.9823 - val_loss : 0.2201 - val_accuracy : 0.9529
round 16 - train_loss : 0.0817 - train_accuracy : 0.9773 - val_loss : 0.1982 - val_accuracy : 0.9545
round 17 - train_loss : 0.0775 - train_accuracy : 0.9820 - val_loss : 0.1813 - val_accuracy : 0.9561
round 18 - train_loss : 0.0538 - train_accuracy : 0.9885 - val_loss : 0.1677 - val_accuracy : 0.9574
round 19 - train_loss : 0.0546 - train_accuracy : 0.9877 - val_loss : 0.1571 - val_accuracy : 0.9582
round 20 - train_loss : 0.0405 - train_accuracy : 0.9910 - val_loss : 0.1485 - val_accuracy

In [2175]:
# fedavg baseline
federater_avg.fit(num_rounds=1, criterion=criterion, num_epochs=1, val_dl=test_dl)

round 1 - train_loss : 0.6275 - train_accuracy : 0.8185 - val_loss : 2.2982 - val_accuracy : 0.1241


In [2176]:
# fedprox baseline
federater_prox.fit(num_rounds=1, criterion=criterion, num_epochs=1, val_dl=test_dl)

round 1 - train_loss : 0.8181 - train_accuracy : 0.7648 - val_loss : 2.2974 - val_accuracy : 0.1303


In [1848]:
from torch.optim.optimizer import Optimizer, required

class FedProx2(Optimizer):
    r"""Implements FedAvg and FedProx. Local Solver can have momentum.
    Nesterov momentum is based on the formula from
    `On the importance of initialization and momentum in deep learning`__.
    Args:
        params (iterable): iterable of parameters to optimize or dicts defining
            parameter groups
        ratio (float): relative sample size of client
        gmf (float): global/server/slow momentum factor
        mu (float): parameter for proximal local SGD
        lr (float): learning rate
        momentum (float, optional): momentum factor (default: 0)
        weight_decay (float, optional): weight decay (L2 penalty) (default: 0)
        dampening (float, optional): dampening for momentum (default: 0)
        nesterov (bool, optional): enables Nesterov momentum (default: False)
    Example:
        >>> optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
        >>> optimizer.zero_grad()
        >>> loss_fn(model(input), target).backward()
        >>> optimizer.step()
    __ http://www.cs.toronto.edu/%7Ehinton/absps/momentum.pdf
    .. note::
        The implementation of SGD with Momentum/Nesterov subtly differs from
        Sutskever et. al. and implementations in some other frameworks.
        Considering the specific case of Momentum, the update can be written as
        .. math::
                  v = \rho * v + g \\
                  p = p - lr * v
        where p, g, v and :math:`\rho` denote the parameters, gradient,
        velocity, and momentum respectively.
        This is in contrast to Sutskever et. al. and
        other frameworks which employ an update of the form
        .. math::
             v = \rho * v + lr * g \\
             p = p - v
        The Nesterov version is analogously modified.
    """

    def __init__(self, params, ratio, gmf, lr=required, momentum=0, dampening=0,
                 weight_decay=0, nesterov=False, variance=0, mu=0):
        
        self.gmf = gmf
        self.ratio = ratio
        self.itr = 0
        self.a_sum = 0
        self.mu = mu


        if lr is not required and lr < 0.0:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if momentum < 0.0:
            raise ValueError("Invalid momentum value: {}".format(momentum))
        if weight_decay < 0.0:
            raise ValueError("Invalid weight_decay value: {}".format(weight_decay))

        defaults = dict(lr=lr, momentum=momentum, dampening=dampening,
                        weight_decay=weight_decay, nesterov=nesterov, variance=variance)
        if nesterov and (momentum <= 0 or dampening != 0):
            raise ValueError("Nesterov momentum requires a momentum and zero dampening")
        super(FedProx2, self).__init__(params, defaults)


    def __setstate__(self, state):
        super(FedProx, self).__setstate__(state)
        for group in self.param_groups:
            group.setdefault('nesterov', False)

    def step(self, closure=None):
        """Performs a single optimization step.
        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            weight_decay = group['weight_decay']
            momentum = group['momentum']
            dampening = group['dampening']
            nesterov = group['nesterov']
            

            for p in group['params']:
                if p.grad is None:
                    continue
                d_p = p.grad.data

                if weight_decay != 0:
                    d_p.add_(weight_decay, p.data)
                
                param_state = self.state[p]
                if 'old_init' not in param_state:
                    param_state['old_init'] = torch.clone(p.data).detach()

                if momentum != 0:
                    if 'momentum_buffer' not in param_state:
                        buf = param_state['momentum_buffer'] = torch.clone(d_p).detach()
                    else:
                        buf = param_state['momentum_buffer']
                        buf.mul_(momentum).add_(1 - dampening, d_p)
                    if nesterov:
                        d_p = d_p.add(momentum, buf)
                    else:
                        d_p = buf

                # apply proximal update
                d_p.add_(self.mu, p.data - param_state['old_init'])
                p.data.add_(-group['lr'], d_p)

        return loss

    def average(self):
        param_list = []
        for group in self.param_groups:
            for p in group['params']:
                p.data.mul_(self.ratio)
                param_list.append(p.data)

        communicate(param_list, dist.all_reduce)

        for group in self.param_groups:
            for p in group['params']:
                param_state = self.state[p]
                param_state['old_init'] = torch.clone(p.data).detach()
                # Reinitialize momentum buffer
                if 'momentum_buffer' in param_state:
                    param_state['momentum_buffer'].zero_()

In [ ]:
class FedAdam(Optimizer):
    """FederatedAveraging (FedAvg) central server as proposed in "Communication-Efficient Learning 
    of Deep Networks from Decentralized Data" (https://arxiv.org/pdf/1602.05629.pdf)
    
    Parameters
    ----------
    model : 
    dataloader :
    device : str (default='cpu')
    """
    def __init__(self, params, lr=1):
        if lr <= 0.0:
            raise ValueError(f'Invalid learning rate: {lr}')
        self.model = model
        self.lr = lr
        defaults = dict(lr=lr)
        
        super(FedAvg, self).__init__(params, defaults)
        
    def aggregate(self, clients, client_weights):
        global_state = {}
        for k, (client_id, client) in enumerate(clients.items()):
            local_state = client.model.state_dict()
            for layer_name, param in local_state.items():
                if k == 0:
                    global_state[layer_name] = self.lr * param * client_weights[k]
                else:
                    global_state[layer_name] += self.lr * param * client_weights[k]

        self.model.load_state_dict(global_state)
        
    def aggregate(self, clients, client_weights):
        
        for k, (client_id, client) in enumerate(clients.items()):
            client_optimizer = client.optimizer
            for group in client_optimizer.param_groups:
                for p in group['params']:
                    

In [1771]:
# class FedAvg(Optimizer):
#     """FederatedAveraging (FedAvg) central server as proposed in "Communication-Efficient Learning 
#     of Deep Networks from Decentralized Data" (https://arxiv.org/pdf/1602.05629.pdf)
    
#     Parameters
#     ----------
#     model : 
#     dataloader :
#     device : str (default='cpu')
#     """
#     def __init__(self, params, lr=1):
#         if lr <= 0.0:
#             raise ValueError(f'Invalid learning rate: {lr}')
#         self.model = model
#         self.lr = lr
#         defaults = dict(lr=lr)
        
#         super(FedAvg, self).__init__(params, defaults)
        
#     def aggregate(self, clients, client_weights):
# #         layer_names = clients[0].model.state_dict().keys()
#         global_state = {}
#         for k, (client_id, client) in enumerate(clients.items()):
#             local_state = client.model.state_dict()
#             for layer_name, param in local_state.items():
#                 if k == 0:
#                     global_state[layer_name] = self.lr * param * client_weights[k]
#                 else:
#                     global_state[layer_name] += self.lr * param * client_weights[k]

#         self.model.load_state_dict(global_state)
        
#     def aggregate(self, clients, client_weights):
#         for k, (client_id, client) in enumerate(clients.items()):
#             client_optimizer = client.optimizer
#             for group in client_optimizer.param_groups:
#                 for p in group['params']:
                    

In [ ]:
class FedAdam(Optimizer):
    """FederatedAveraging (FedAvg) central server as proposed in "Communication-Efficient Learning 
    of Deep Networks from Decentralized Data" (https://arxiv.org/pdf/1602.05629.pdf)
    
    Parameters
    ----------
    model : 
    dataloader :
    device : str (default='cpu')
    """
    def __init__(self, params, lr=1):
        if lr <= 0.0:
            raise ValueError(f'Invalid learning rate: {lr}')
        self.model = model
        self.lr = lr
        defaults = dict(lr=lr)
        
        super(FedAvg, self).__init__(params, defaults)
        
    def aggregate(self, clients, client_weights):
        global_state = {}
        for k, (client_id, client) in enumerate(clients.items()):
            local_state = client.model.state_dict()
            for layer_name, param in local_state.items():
                if k == 0:
                    global_state[layer_name] = self.lr * param * client_weights[k]
                else:
                    global_state[layer_name] += self.lr * param * client_weights[k]

        self.model.load_state_dict(global_state)
        
    def aggregate(self, clients, client_weights):
        
        for k, (client_id, client) in enumerate(clients.items()):
            client_optimizer = client.optimizer
            for group in client_optimizer.param_groups:
                for p in group['params']:
                    

In [16]:
    
class FedProx(BaseFederater):
    
    def __init__(self, optimizer, clients, client_weights):
        self.optimizer = optimizer
        self.param_groups = self.optimizer.param_groups
    
    def aggregate(self):
        layer_names = self.model.state_dict().keys()
        global_state = {}
        for k, (client_id, client) in enumerate(clients.items()):
            local_state = client.model.state_dict()
            for layer_name in layer_names:
                if k == 0:
                    global_state[layer_name] = local_state[layer_name] * client_weights[k]
                else:
                    global_state[layer_name] += local_state[layer_name] * client_weights[k]

        self.update(global_state)
        
        
        
class FedAvg(Optimizer):
    
    def __init__(self, params, cleints, client_weights):
        self.clients = clients
#         defaults = dict(lr=lr, momentum=momentum)
        super(FedProxGlobal, self).__init__(params, defaults)
        
    def step(self, closure=None):
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()
        
        
        for group in self.param_groups:
            for p in group['params']:
                pass
                
                
    def aggregate(self, clients, client_weights):
        layer_names = self.model.state_dict().keys()
        global_state = {}
        for k, (client_id, client) in enumerate(clients.items()):
            local_state = client.model.state_dict()
            for layer_name in layer_names:
                if k == 0:
                    global_state[layer_name] = local_state[layer_name] * client_weights[k]
                else:
                    global_state[layer_name] += local_state[layer_name] * client_weights[k]

        self.update(global_state)

In [ ]:
# client optimizer
# sever optimizer
# federater

In [1195]:
model = CNN()
clients = get_clients(
    train_ds, 
    method='fedprox',
    num_clients=100,
    is_iid=False,
    shard_size=300,
    batch_size=50,
    num_workers=0,
    device=device
)
fed_cls = FedProx
federater = fed_cls(model, 
                   clients=clients,
                   optim_cls=optim_cls,
                   optim_params=optim_params,
                   criterion=criterion, 
                   C=0.1,
                    mu=1000,
                   output_dir='mu1'
                  )

In [1196]:
# 1
federater.fit(num_rounds=10, num_epochs=1, val_dl=test_dl)

round 1 - train_loss : 78.3555 - train_accuracy : 0.8820 - train_xent_loss : 0.9399 - train_weights_delta : 0.0808 - val_loss : 2.3053 - val_accuracy : 0.1034
round 2 - train_loss : 81.7870 - train_accuracy : 0.8213 - train_xent_loss : 0.9839 - train_weights_delta : 0.0840 - val_loss : 2.3001 - val_accuracy : 0.1042
round 3 - train_loss : 93.3500 - train_accuracy : 0.8510 - train_xent_loss : 0.8035 - train_weights_delta : 0.0922 - val_loss : 2.2993 - val_accuracy : 0.1529
round 4 - train_loss : 101.7852 - train_accuracy : 0.8333 - train_xent_loss : 0.7750 - train_weights_delta : 0.0989 - val_loss : 2.3005 - val_accuracy : 0.1253
round 5 - train_loss : 106.8303 - train_accuracy : 0.8857 - train_xent_loss : 0.6514 - train_weights_delta : 0.1007 - val_loss : 2.3055 - val_accuracy : 0.1460
round 6 - train_loss : 114.3917 - train_accuracy : 0.8423 - train_xent_loss : 0.6932 - train_weights_delta : 0.1076 - val_loss : 2.2933 - val_accuracy : 0.1507
round 7 - train_loss : 126.1735 - train_acc

In [1163]:
# 1
federater.fit(num_rounds=50, num_epochs=1, val_dl=test_dl)

round 1 - train_loss : 1.0243 - train_accuracy : 0.8240 - train_xent_loss : 0.9379 - train_weights_delta : 0.0888 - val_loss : 2.3261 - val_accuracy : 0.1138
round 2 - train_loss : 0.9458 - train_accuracy : 0.8500 - train_xent_loss : 0.8507 - train_weights_delta : 0.0953 - val_loss : 2.3223 - val_accuracy : 0.1138
round 3 - train_loss : 0.8591 - train_accuracy : 0.8500 - train_xent_loss : 0.7551 - train_weights_delta : 0.1012 - val_loss : 2.3194 - val_accuracy : 0.1162
round 4 - train_loss : 0.7957 - train_accuracy : 0.8503 - train_xent_loss : 0.6841 - train_weights_delta : 0.1058 - val_loss : 2.3144 - val_accuracy : 0.1673
round 5 - train_loss : 0.7910 - train_accuracy : 0.8477 - train_xent_loss : 0.6673 - train_weights_delta : 0.1152 - val_loss : 2.3000 - val_accuracy : 0.1278
round 6 - train_loss : 0.7412 - train_accuracy : 0.8677 - train_xent_loss : 0.6166 - train_weights_delta : 0.1148 - val_loss : 2.2938 - val_accuracy : 0.1147
round 7 - train_loss : 0.6914 - train_accuracy : 0.8

In [ ]:
# import torch.nn.functional as F


# class FedProxLoss(nn.Module):
#     """
#     Empirically, we observe that increasing µ leads to smaller dissimilarity among local functions
#     Fk, and that the dissimilarity metric is consistent with the
#     training loss.
#     """
#     def __init__(self, mu, weights_initial):
#         super(FedProxLoss, self).__init__()
#         self.mu = mu
#         self.weights_initial = weights_initial
#         self._layer_names = list(weights_initial.keys())
        
        
#     def forward(self, pred, target, weights_new):
#         loss = 0
#         xent_loss = F.cross_entropy(pred, target)
# #         loss = xent_loss
#         weights_delta = []
#         for layer in self._layer_names:
#             layer_delta = torch.sum(torch.pow(self.weights_initial[layer] - weights_new[layer], 2))
#             loss += self.mu / 2 * layer_delta
#             weights_delta.append(layer_delta)
# #             weights_delta.append(
# #                 torch.sum(torch.pow(self.weights_initial[layer] - weights_new[layer], 2))
# #             )
#         weights_delta = torch.sum(torch.stack(weights_delta))
#         loss += xent_loss
# #         print(xent_loss)
# #         loss = xent_loss + self.mu / 2. * weights_delta
#         return loss, xent_loss, weights_delta
# #         loss += 